In [2]:
import re
from pathlib import Path
from io import BytesIO, StringIO

import pandas as pd
import numpy as np
import PIL
import PIL.Image

In [5]:
F_BALA_IMG_SRC = Path("/Users/nonlogical/Documents/Arduino/libraries/M5Bala2/src/bala_img.h")
F_IMG_JPG = Path("data/img.jpg")
F_IMG_JB_JPG = Path("data/img_jb.jpg")
F_IMG_JB_PNG = Path("data/img_jb.png")

In [81]:
def bala_img_decode():
	img_src_path = F_BALA_IMG_SRC
	out_img = F_IMG_JPG

	img_src_data_r = img_src_path.read_text()
	img_src_data = []
	octal_re = re.compile(r'(0x[a-f0-9]{2}),')

	for l in img_src_data_r.splitlines():
		matches = octal_re.finditer(l)
		for m in matches:
			img_word = int(m.groups()[0], 16)
			img_src_data.append(img_word)

	out_img.write_bytes(np.array(img_src_data, dtype=np.uint8).tobytes())

In [12]:
file_tpl = """
#ifndef __BALA_IMG_H
#define __BALA_IMG_H

#include "Arduino.h"

const uint8_t bala_img[{data_len}] = {{
{data}
}};

#endif
"""

def bala_img_encode():
	in_png = PIL.Image.open(F_IMG_JB_PNG)
	in_png.save(F_IMG_JB_JPG, format='JPEG', subsampling=0, quality=100)
	in_img = F_IMG_JB_JPG
	out_src = F_BALA_IMG_SRC
	in_img_b = bytearray(in_img.read_bytes())
	encoded_c_data = StringIO()
	for i, b in enumerate(in_img_b):
		l_begin = i%16 == 0
		l_end = (i+1)%16 == 0
		end = ", "
		prefix = ""
		if l_begin:
			prefix = "    "
		if l_end:
			end = ",\n"
		print("%s0x%s" % (prefix, hex(b)[2:].rjust(2, "0")), end=end, file=encoded_c_data)

	encoded_c_data.seek(0)
	out_src.write_text(file_tpl.format(data=encoded_c_data.read(), data_len=len(in_img_b)))
	print("OK")

In [13]:
bala_img_encode()

OK
